# Solar Power Prediction
El presente documento tiene por objetivo realizar un estudio sobre la predicción de la producción de una planta solar en India. Los datos corresponden a 34 días. Intentaremos responder la siguientes preguntas:
- ¿Podemos predecir la generación eléctrica para los próximos días?
- ¿Podemos identificar la necesidad de limpieza del panel?
- ¿Podemos identificar alguna falla o baja en el rendimiento de los paneles?

Antes de partir con el análisis, importaremos los módulos necesarios. Pandas, Seabron y Sckit Learn serán nuestros paquetes principales.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

A continuación se hará la lectura del archivo que contiene la información de la planta de generación. En ella encontraremos las siguientes columnas:
- **DATE_TIME**: Dato temporal, indica el tiempo en el cual se realizó la medición.
- **PLANT_ID**: La identificación de la planta, por ende, en todo el archivo es igual.
- **SOURCE_KEY**: ID del inversor.
- **DC_POWER**: Cantidad de corriente directa generada en un periodo de 15 minutos, medida en kW.
- **AC_POWER**: Cantidad de corriente alterna generada en un periodo de 15 minutos, medida en kW.
- **DAILY_YIELD**: Cantidad de energía diaria acumulada hasta ese punto en el tiempo.
- **TOTAL_YIELD**: Cantidad de energía acumulada en ese inversor, hasta ese punto en el tiempo.

In [4]:
df = pd.read_csv('archive/Plant_1_Generation_Data.csv')
df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


A continuación se hará la lectura del archivo que contiene la información de la planta de generación. En ella encontraremos las siguientes columnas:
- **DATE_TIME**: Dato temporal, indica el tiempo en el cual se realizó la medición.
- **PLANT_ID**: La identificación de la planta, por ende, en todo el archivo es igual.
- **SOURCE_KEY**: ID del inversor.
...


In [5]:
df_w = pd.read_csv('archive/Plant_1_Weather_Sensor_Data.csv')
df_w.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [22]:
#Transformamos la columna DATE_TIME en ambos dataframe en datos temporales
#Notar que ambos tienen un formato temporal distinto
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], format='%d-%m-%Y %H:%M')
df_w['DATE_TIME'] = pd.to_datetime(df_w['DATE_TIME'], format='%Y%m%d %H:%M:%S')

Verificamos que en ambos dataframe no hayan elementos no nulos. Además, chequeamos que la columna DATE_TIME efectivamente sea un dato temporal.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68778 entries, 0 to 68777
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE_TIME    68778 non-null  datetime64[ns]
 1   PLANT_ID     68778 non-null  int64         
 2   SOURCE_KEY   68778 non-null  object        
 3   DC_POWER     68778 non-null  float64       
 4   AC_POWER     68778 non-null  float64       
 5   DAILY_YIELD  68778 non-null  float64       
 6   TOTAL_YIELD  68778 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 3.7+ MB


In [27]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3182 entries, 0 to 3181
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            3182 non-null   datetime64[ns]
 1   PLANT_ID             3182 non-null   int64         
 2   SOURCE_KEY           3182 non-null   object        
 3   AMBIENT_TEMPERATURE  3182 non-null   float64       
 4   MODULE_TEMPERATURE   3182 non-null   float64       
 5   IRRADIATION          3182 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 149.3+ KB
